In [ ]:
import numpy as np

input_data = """
C     -0.00071950    0.00012936    1.41893938
C     -0.80266622   -0.90375612    2.11523059
C      0.80095602    0.90407222    2.11738416
C     -0.79789402   -0.91529281    3.50349487
H     -1.42549155   -1.60507770    1.57253396
C      0.79974743    0.89927078    3.50414604
H      1.41695052    1.61334256    1.57739263
C      0.00728328   -0.01286226    4.19845680
H     -1.42239916   -1.61754859    4.04555024
H      1.41376747    1.59479538    4.06683164
C     -0.22164038   -1.03276771    6.33154283
C      0.40523173   -2.25310029    6.08337740
C     -1.06569645   -0.87295622    7.42561663
C      0.16815717   -3.32289291    6.94026619
H      1.06783003   -2.35540253    5.22924748
C     -1.28647090   -1.94873083    8.28037376
H     -1.53263484    0.09292739    7.59101598
C     -0.67575072   -3.17689974    8.03948424
H      0.65500462   -4.27539441    6.74937551
H     -1.94449833   -1.82486134    9.13609302
H     -0.85381228   -4.01586234    8.70584875
O      0.02397012    0.08081969    5.56186226
N      0.00000000    0.00000000    0.00000000
C      0.00001711    1.16537840   -0.78999403
C      0.00033750   -1.16414734   -0.78933740
C      0.00034055    0.69439670   -2.20054876
O      0.00166705    2.30315146   -0.38703683
C     -0.00019653   -0.69482562   -2.20203488
O     -0.00497514   -2.30280394   -0.38754053
C      0.00482247    1.42354364   -3.37386233
C      0.01183197   -1.42816754   -3.37524492
C      0.00075958    0.68188327   -4.55732887
H      0.03097269    2.50869215   -3.37936280
C      0.01113838   -0.71454300   -4.57516145
H      0.02584809   -2.51329639   -3.35659044
S     -0.01498589    1.55969587   -6.11719314
H      0.03952439   -1.22822630   -5.53131687
C     -1.75926497    1.85944591   -6.38923532
O      0.43335801    0.62710885   -7.14832566
O      0.62119881    2.85588854   -5.89590687
C     -2.51626765    0.86955708   -7.01896989
C     -2.30396490    3.07341355   -5.96494773
C     -3.85919873    1.14596762   -7.18945176
H     -2.06730432   -0.05404763   -7.37082414
C     -3.66256194    3.33279965   -6.15278685
H     -1.65144932    3.81253323   -5.51011614
C     -4.41872623    2.34815611   -6.76376429
C     -4.93264601    0.31723790   -7.81241010
H     -4.11230997    4.26956078   -5.83912913
C     -5.87437341    2.33348046   -7.09952074
N     -6.08303278    1.09964978   -7.71351103
O     -4.85545885   -0.78518499   -8.29667938
O     -6.70645491    3.18346013   -6.89605827
H     -6.98503072    0.79902546   -8.05475885
"""

lines = input_data.strip().split('\n')

atomic_masses = {
    'H': 1.008,
    'He': 4.0026,
    'Li': 6.94,
    'Be': 9.0122,
    'B': 10.81,
    'C': 12.011,
    'N': 14.007,
    'O': 15.999,
    'F': 18.998,
    'Ne': 20.180,
    'S': 32.065
    # 可根據需要添加更多元素
}

positions = []
masses = []

for line in lines:
    parts = line.split()
    symbol = parts[0]
    coords = list(map(float, parts[1:4]))
    positions.append(coords)
    masses.append(atomic_masses[symbol])

def calculate_radius_of_gyration(positions, masses):
    """
    Calculate the radius of gyration for a group of atoms.

    Parameters:
    positions (np.ndarray): An N x 3 array of atomic positions (x, y, z).
    masses (np.ndarray): A 1D array of atomic masses.

    Returns:
    float: The radius of gyration.
    """
    positions = np.array(positions)
    masses = np.array(masses)
    total_mass = np.sum(masses)
    com = np.sum(positions.T * masses, axis=1) / total_mass
    squared_distances = np.sum((positions - com) ** 2, axis=1)
    R_g = np.sqrt(np.sum(squared_distances * masses) / total_mass)
    return R_g

def calculate_squared_radius_of_gyration(positions, masses):
    """
    Calculate the squared radius of gyration for a group of atoms.

    Parameters:
    positions (np.ndarray): An N x 3 array of atomic positions (x, y, z).
    masses (np.ndarray): A 1D array of atomic masses.

    Returns:
    float: The squared radius of gyration (R_g^2).
    """
    positions = np.array(positions)
    masses = np.array(masses)
    total_mass = np.sum(masses)
    com = np.sum(positions.T * masses, axis=1) / total_mass
    squared_distances = np.sum((positions - com) ** 2, axis=1)
    R_g_squared = np.sum(squared_distances * masses) / total_mass
    return R_g_squared

def compute_gyration_tensor_and_shape_parameters(positions, box_length=None):
    N = len(positions)

    # Calculate the center of mass
    R_CM = np.mean(positions, axis=0)

    # Adjust positions for periodic boundaries if box_length is provided
    adjusted_positions = positions - R_CM
    if box_length:
        adjusted_positions -= box_length * np.round(adjusted_positions / box_length)

    # Compute the gyration tensor
    G = np.zeros((3, 3))
    for i in range(N):
        r = adjusted_positions[i]
        G += np.outer(r, r)
    G /= N

    # Calculate eigenvalues of the gyration tensor
    eigenvalues = np.linalg.eigvalsh(G)
    Ix, Iy, Iz = np.sort(eigenvalues)

    # Compute shape parameters
    b = Iz - 0.5 * (Iy + Ix)
    c = Iy - Ix
    k = (1.5 * (Ix**2 + Iy**2 + Iz**2) / (Ix + Iy + Iz)**2) - 0.5

    return Ix, Iy, Iz, b, c, k

# Calculate both R_g and R_g^2
R_g = calculate_radius_of_gyration(positions, masses)
R_g_squared = calculate_squared_radius_of_gyration(positions, masses)

# Print the results
print(f"The radius of gyration is: {R_g}")
print(f"Square of the radius of gyration: {R_g**2}")

# No periodic boundary conditions assumed here (box_length=None)
Ix, Iy, Iz, b, c, k = compute_gyration_tensor_and_shape_parameters(positions)

print(f"Eigenvalues of the gyration tensor: Ix = {Ix}, Iy = {Iy}, Iz = {Iz}")
print(f"Asphericity (b): {b}")
print(f"Acylindricity (c): {c}")
print(f"Relative Shape Anisotropy (k): {k}")

The radius of gyration is: 5.9713608785187855
The squared radius of gyration is: 35.657150741504644
Square of the radius of gyration: 35.657150741504644
Eigenvalues of the gyration tensor: Ix = 2.2754055455028093, Iy = 3.1354349279349822, Iz = 34.05393263744088
Asphericity (b): 31.348512400721983
Acylindricity (c): 0.860029382432173
Relative Shape Anisotropy (k): 0.631334812466648
